In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from datetime import datetime
import pandas as pd
import traceback

# Chromeドライバのセットアップ（事前にインストール必要）
driver = webdriver.Chrome()

# 読み込むCSVやDataFrameの準備
df = pd.read_csv("cleaned_unique_titles.csv")  # タイトル列が "タイトル"
df["発売日"] = ""
df["備考"] = ""
total = len(df)
not_found_titles = []

for idx, row in df.head(5).iterrows():
    title = row["タイトル"]
    print(f"[{idx+1}/{total}] 検索中：{title}")

    try:
        # 1. トップページ → 検索実行
        driver.get("https://www.4gamer.net/")
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "word")))

        search_box = driver.find_element(By.NAME, "word")
        search_box.clear()
        search_box.send_keys(title)
        search_box.send_keys(Keys.RETURN)

        # 2. 検索結果を取得
        try:
            WebDriverWait(driver, 10).until(
                EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'h2 > a[href^="/games/"]'))
            )
            links = driver.find_elements(By.CSS_SELECTOR, 'h2 > a[href^="/games/"]')
        except TimeoutException:
            print(" → 検索結果が見つかりません（タイムアウト）")
            not_found_titles.append(title)
            continue

        game_links = [link for link in links if "/games/" in (link.get_attribute("href") or "")]
        if not game_links:
            print(" → ゲーム紹介ページが見つかりません")
            not_found_titles.append(title)
            continue

        # 3. 詳細ページをたどって発売日・備考を取得
        release_dates = []

        for link in game_links:
            try:
                url = link.get_attribute("href")
                if not url.startswith("http"):
                    url = "https://www.4gamer.net" + url

                driver.get(url)
                WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.TAG_NAME, "th"))
                )

                th_list = driver.find_elements(By.TAG_NAME, "th")

                for th in th_list:
                    # 発売日の取得
                    if "発売日" in th.text:
                        try:
                            td = th.find_element(By.XPATH, "./following-sibling::td[2]")
                            date_text = td.text.strip()
                            if not date_text or date_text in ["-", "："]:
                                continue
                            try:
                                parsed = datetime.strptime(date_text, "%Y/%m/%d")
                                release_dates.append((parsed, date_text))
                            except ValueError:
                                continue
                        except:
                            continue

                    # 備考の取得
                    if "備考" in th.text:
                        try:
                            td = th.find_element(By.XPATH, "./following-sibling::td[2]")
                            remark_items = td.find_elements(By.TAG_NAME, "a")
                            remarks = [item.text.strip() for item in remark_items if item.text.strip()]
                            if remarks:
                                df.at[idx, "備考"] = ", ".join(remarks)
                        except:
                            continue

            except Exception:
                continue

        # 4. 最も古い発売日を採用
        if release_dates:
            oldest = min(release_dates, key=lambda x: x[0])[1]
            df.at[idx, "発売日"] = oldest
            print(f" → 最も古い発売日：{oldest}")
        else:
            print(" → 発売日が見つかりません")
            not_found_titles.append(title)

    except Exception:
        print(" → メイン処理エラー")
        traceback.print_exc()
        not_found_titles.append(title)
        continue

# 結果をCSVに保存
df.to_csv("4gamer_output.csv", index=False, encoding="utf-8-sig")

# 終了処理
driver.quit()
print("\n完了しました。取得できなかったタイトル数:", len(not_found_titles))


[1/393] 検索中：スーパーガンダムロワイヤル
 → 最も古い発売日：2015/10/16
[2/393] 検索中：武器よさらば
 → 最も古い発売日：2017/03/30
[3/393] 検索中：スーパーロボット大戦X-Ω
 → 最も古い発売日：2015/10/05
[4/393] 検索中：FINAL FANTASY GRANDMASTERS
 → 最も古い発売日：2015/10/01
[5/393] 検索中：ドラゴンプロジェクト
 → 最も古い発売日：2016/06/03

完了しました。取得できなかったタイトル数: 0


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from datetime import datetime
import pandas as pd
import traceback
import re

# Chromeドライバのセットアップ（事前にインストール必要）
driver = webdriver.Chrome()

# CSV読み込みと初期化
df = pd.read_csv("cleaned_unique_titles.csv")  # タイトル列が "タイトル"
df["発売日"] = ""
df["備考"] = ""
total = len(df)
not_found_titles = []

# 最初の5件のみ処理
for idx, row in df.head(5).iterrows():
    title = row["タイトル"]
    print(f"[{idx+1}/{total}] 検索中：{title}")

    # --- 前半 try: 検索操作 ---
    try:
        # トップページから検索実行
        driver.get("https://www.4gamer.net/")
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "word")))

        search_box = driver.find_element(By.NAME, "word")
        search_box.clear()
        search_box.send_keys(title)
        search_box.send_keys(Keys.RETURN)

        # 検索結果を取得（リンクのhrefだけ抽出しておく）
        WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'h2 > a[href^="/games/"]'))
        )
        links = driver.find_elements(By.CSS_SELECTOR, 'h2 > a[href^="/games/"]')
        game_links = []

        for link in links:
            href = link.get_attribute("href")
            if href and "/games/" in href:
                game_links.append(href)

        if not game_links:
            print(" → ゲーム紹介ページが見つかりません")
            not_found_titles.append(title)
            continue

    except Exception:
        print(" → 検索処理中にエラーが発生しました")
        traceback.print_exc()
        not_found_titles.append(title)
        continue

    # --- 後半 try: 詳細ページ処理 ---
    try:
        release_dates = []

        for url in game_links:
            if not url.startswith("http"):
                url = "https://www.4gamer.net" + url

            driver.get(url)
            WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.TAG_NAME, "th"))
            )

            th_list = driver.find_elements(By.TAG_NAME, "th")

            # 発売日
            for th in th_list:
                if "発売日" in th.text:
                    td_list = th.find_elements(By.XPATH, "./following-sibling::td")
                    if len(td_list) >= 2:
                        date_text = td_list[1].text.strip()
                        if date_text and date_text not in ["-", "："] and re.match(r"\d{4}/\d{2}/\d{2}", date_text):
                            parsed = datetime.strptime(date_text, "%Y/%m/%d")
                            release_dates.append((parsed, date_text))
                            break

            # 備考
            for th in th_list:
                if "備考" in th.text:
                    td_list = th.find_elements(By.XPATH, "./following-sibling::td")
                    if len(td_list) >= 2:
                        a_tags = td_list[1].find_elements(By.TAG_NAME, "a")
                        remarks = [a.text.strip() for a in a_tags if a.text.strip()]
                        if remarks:
                            df.at[idx, "備考"] = ", ".join(remarks)
                    break

    except Exception:
        print(" → 詳細ページ取得中にエラーが発生しました")
        traceback.print_exc()
        not_found_titles.append(title)
        continue

    # 発売日決定
    if release_dates:
        oldest = min(release_dates, key=lambda x: x[0])[1]
        df.at[idx, "発売日"] = oldest
        print(f" → 最も古い発売日：{oldest}")
        if df.at[idx, "備考"]:
            print(f" → 備考：{df.at[idx, '備考']}")
    else:
        print(" → 発売日が見つかりません")
        not_found_titles.append(title)

# 結果保存
df.to_csv("4gamer_output.csv", index=False, encoding="utf-8-sig")

# 終了処理
driver.quit()
print("\n完了しました。取得できなかったタイトル数:", len(not_found_titles))

[1/393] 検索中：スーパーガンダムロワイヤル
 → 最も古い発売日：2015/10/16
 → 備考：アニメ/コミック, 無料
[2/393] 検索中：武器よさらば
 → 最も古い発売日：2017/03/30
 → 備考：無料, プレイ人数：1人
[3/393] 検索中：スーパーロボット大戦X-Ω
 → 最も古い発売日：2015/10/05
 → 備考：アニメ/コミック, ロボット, 戦争物, 無料
[4/393] 検索中：FINAL FANTASY GRANDMASTERS
 → 最も古い発売日：2015/10/01
 → 備考：日本, ファイナルファンタジー, 無料, ファンタジー
[5/393] 検索中：ドラゴンプロジェクト
 → 最も古い発売日：2016/06/03
 → 備考：プレイ人数：1～4人, 協力プレイ, ファンタジー, 無料, 日本

完了しました。取得できなかったタイトル数: 0
